In [ ]:
Парсер товаров с сайта sbermarket.ru/lenta
Код частично удален, полный код можно запросить тут: 9m6@bk.ru 

In [2]:
import requests
from bs4 import BeautifulSoup
import csv
import os
import re

In [54]:
URL = 'https://sbermarket.ru/lenta/c/dieshievlie-chiem-na-polkie/bakalieia?discounted=true'
HEADERS = {'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:87.0) Gecko/20100101 Firefox/87.0', 'accept': '*/*'}
HOST = 'https://sbermarket.ru'
FILE = 'lenta.csv'

In [4]:
def get_html(url, params=None):
    request = requests.get(url, headers=HEADERS, params=params)
    return request

In [39]:
def get_pages_count(html):
    soup = BeautifulSoup(html, 'html.parser')
    pagination = soup.find_all('a', class_='gradient')[1:-1]
    if pagination:
        return int(pagination[-1].get_text())
    else:
        return 1

In [51]:
def get_content(html):
    soup = BeautifulSoup(html, 'html.parser')
    items = soup.find_all('li', class_='product')
    schools = []
    for item in items:
        schools.append({
            'title': item.find('p', class_='product__title').get_text(strip=True), 
            'sale': int(re.sub("-|%| ","",item.find('div', class_='price-discount-info__item price-discount-info__item--percent').get_text(strip=True))),
            'price': re.sub("-|₽| ","",item.find('div', class_='price price--default').get_text(strip=True)),
            'site': HOST + item.find('a', class_='product__link').get('href')
        })
    return schools

In [49]:
def save_file(items, path):
    with open(path, 'w', newline='') as file:
        writer = csv.writer(file, delimiter=';')
        writer.writerow(['Продукт', 'Скидка', 'Цена','Ссылка'])
        for item in items:
            writer.writerow([item['title'], item['sale'], item['price'], item['site']])